# DATA Exploration

# 1. Discovering Data

In [31]:
#import sklearn as skl
import pandas as pd

df = pd.read_csv("SBAnational.csv", low_memory=False)
print(f"{df.shape[0]} lignes")
df.describe()

899164 lignes


,LoanNr_ChkDgt,Zip,NAICS,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural
count,8.991640e+05,899164.000000,899164.000000,899164.000000,899164.000000,899028.000000,899164.000000,899164.000000,899164.000000,899164.000000
mean,4.772612e+09,53804.391241,398660.950146,110.773078,11.411353,1.280404,8.430376,10.797257,2753.725933,0.757748
std,2.538175e+09,31184.159152,263318.312760,78.857305,74.108196,0.451750,236.688165,237.120600,12758.019136,0.646436
min,1.000014e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2.589758e+09,27587.000000,235210.000000,60.000000,2.000000,1.000000,0.000000,0.000000,1.000000,0.000000
50%,4.361439e+09,55410.000000,445310.000000,84.000000,4.000000,1.000000,0.000000,1.000000,1.000000,1.000000
75%,6.904627e+09,83704.000000,561730.000000,120.000000,10.000000,2.000000,1.000000,4.000000,1.000000,1.000000
max,9.996003e+09,99999.000000,928120.000000,569.000000,9999.000000,2.000000,8800.000000,9500.000000,99999.000000,2.000000


In [32]:
df.head(2)

,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
0,1000014003,ABC HOBBYCRAFT,EVANSVILLE,IN,47711,FIFTH THIRD BANK,OH,451120,28-Feb-97,1997,...,N,Y,NaN,28-Feb-99,"$60,000.00",$0.00,P I F,$0.00,"$60,000.00","$48,000.00"
1,1000024006,LANDMARK BAR & GRILLE (THE),NEW PARIS,IN,46526,1ST SOURCE BANK,IN,722410,28-Feb-97,1997,...,N,Y,NaN,31-May-97,"$40,000.00",$0.00,P I F,$0.00,"$40,000.00","$32,000.00"


# 1. 1. Column selection

In [33]:
for column_name in df.columns.values :
    print(f"{column_name} : {df[column_name].dtype})")
     

LoanNr_ChkDgt : int64)
Name : object)
City : object)
State : object)
Zip : int64)
Bank : object)
BankState : object)
NAICS : int64)
ApprovalDate : object)
ApprovalFY : object)
Term : int64)
NoEmp : int64)
NewExist : float64)
CreateJob : int64)
RetainedJob : int64)
FranchiseCode : int64)
UrbanRural : int64)
RevLineCr : object)
LowDoc : object)
ChgOffDate : object)
DisbursementDate : object)
DisbursementGross : object)
BalanceGross : object)
MIS_Status : object)
ChgOffPrinGr : object)
GrAppv : object)
SBA_Appv : object)


Explanation of columns :

LoanNr_ChkDgt : primary key,  excluded

Name : excluded

City : excluded

- ${X_1}$ State : it seems the state has an influence on MIS_Status : included

Zip (numeric) : exluded

Bank : excluded

BankState : excluded

- ${X_2}$ NAICS (numeric) : included (2 first chars )

ApprovalDate (date) : excluded

- ${X_3}$  ApprovalFY not sure of inclusion : allows you to keep information on the date - if it is usefull
- ${X_4}$ Term (numeric): included : month number, to be linked to the GrAppv
- ${X_5}$ NoEmp, included : number of employees
- ${X_6}$ NewExist, included 
- ${X_7}$ CreateJob, included 
- ${X_{8}}$ RetainedJob, included 
- ${X_{9}}$ FranchiseCode
- ${X_{10}}$ UrbanRural, included 
- ${X_{11}}$ RevLineCr : included
- ${X_{12}}$ LowDoc : included - not sure it helps : if amount < 150 000)

ChgOffDate : exluded  <= explicit end on simulation 

DisbursementDate : excluded

DisbursementGross : excluded

BalanceGross : excluded - 14 valeurs seulement différentes de zéro  ?

- ${Target} = {Y}$  = MIS_Status : INCLUDED !!!

ChgOffPrinGr : excluded

- ${X_{13}}$  GrAppv : included instead of DisbursementGross (often the same number)

- ${X_{14}}$ SBA_Appv : included

In [34]:
values = df["MIS_Status"].unique()
print(values)
value_counts = df["MIS_Status"].value_counts()
print(value_counts)
print(f"Total lines = {df.shape[0]}")

print (f"lines to remove: {df.shape[0] - value_counts["P I F"] - value_counts["CHGOFF"]}") 

['P I F' 'CHGOFF' nan]
MIS_Status
P I F     739609
CHGOFF    157558
Name: count, dtype: int64
Total lines = 899164
lines to remove: 1997


# 1. 2. doing anything
(Exploiting Target Leaking)

In [35]:
df_Missing_MIS_Status = df[(df["MIS_Status"].isna()) & (df["ChgOffDate"].isna())]
print(f"{df_Missing_MIS_Status.shape[0]} lines probably in 'P I F' Status")
df_Missing_MIS_Status.head(4)


1736 lines probably in 'P I F' Status


,LoanNr_ChkDgt,Name,City,State,Zip,Bank,BankState,NAICS,ApprovalDate,ApprovalFY,...,RevLineCr,LowDoc,ChgOffDate,DisbursementDate,DisbursementGross,BalanceGross,MIS_Status,ChgOffPrinGr,GrAppv,SBA_Appv
343,1002585010,"Chinaberry, LTD",Saratoga Springs,NY,12866,KEYBANK NATIONAL ASSOCIATION,OH,0,18-May-98,1998,...,0,N,NaN,30-Jun-98,$474.00,$0.00,NaN,$0.00,"$30,000.00","$15,000.00"
738,1005264007,AMERICAN CERTIFIED HOME INSPEC,BOISE,ID,83703,WELLS FARGO BANK NATL ASSOC,SD,0,3-Mar-97,1997,...,T,N,NaN,30-Apr-00,"$2,585.00",$0.00,NaN,$0.00,"$10,000.00","$5,000.00"
833,1005845000,SPROUT KIDS INC.,HUNTINGTON,NY,11743,JPMORGAN CHASE BANK NATL ASSOC,IL,424330,3-Mar-03,2003,...,N,N,NaN,31-Mar-03,"$1,276.00",$0.00,NaN,$0.00,"$25,000.00","$12,500.00"
897,1006205009,CHINWE NDUKA MD FAMILY PRACTIC,EL PASO,TX,79905,JPMORGAN CHASE BANK NATL ASSOC,IL,621111,9-Apr-03,2003,...,N,N,NaN,30-Apr-03,$803.00,$0.00,NaN,$0.00,"$40,000.00","$20,000.00"


In [36]:
print(df.shape[0])
df_exp = df[(df["MIS_Status"] =="P I F") | (df["MIS_Status"] =="CHGOFF")]
print(df_exp.shape[0])


899164
897167


# 2. Column conditionning

In [37]:
df["LowDoc"].value_counts()

LowDoc
N    782822
Y    110335
0      1491
C       758
S       603
A       497
R        75
1         1
Name: count, dtype: int64

In [38]:
# df["Term"].value_counts()
df["Term"].describe()

count    899164.000000
mean        110.773078
std          78.857305
min           0.000000
25%          60.000000
50%          84.000000
75%         120.000000
max         569.000000
Name: Term, dtype: float64

In [39]:
569 / 12

47.416666666666664

In [40]:
df["FranchiseCode"].describe()

count    899164.000000
mean       2753.725933
std       12758.019136
min           0.000000
25%           1.000000
50%           1.000000
75%           1.000000
max       99999.000000
Name: FranchiseCode, dtype: float64

In [41]:
df["BalanceGross"].value_counts()

BalanceGross
$0.00           899150
$12,750.00           1
$827,875.00          1
$25,000.00           1
$37,100.00           1
$43,127.00           1
$84,617.00           1
$1,760.00            1
$115,820.00          1
$996,262.00          1
$395,476.00          1
$41,509.00           1
$600.00              1
$9,111.00            1
$96,908.00           1
Name: count, dtype: int64

In [42]:
#df["ApprovalDate"].head()
years = df["ApprovalDate"].value_counts().keys()
years = list(map (lambda x : str(x).split('-')[-1], years))
years = set(years)

#years min = 1961, max = 2014
# on peut prendre 1960 -> 2015 par exemple


In [43]:
ok_values = 739609
ko_values = 157558

ko_values/(ok_values+ko_values)

0.17561724851671986

In [44]:
import datetime as dt
from typing import cast

def specific_convert_date(datestring : str) -> dt.date:
    date_val = dt.date( 1989, 11, 9)
    try :
        date_val = dt.datetime.strptime(datestring, '%d-%b-%y').date()
    except Exception as ex :
        print(ex)

    return date_val

In [1]:
df["ApprovalFY"].value_counts()
# df0["ApprovalYear2"] = df0["ApprovalYear"].apply(lambda x : specific_convert_date(str(x)))
# df0["ApprovalYear2"].describe()

NameError: name 'df' is not defined

In [46]:
df["ChgOffDate"].describe()

count        162699
unique         6448
top       13-Mar-10
freq            734
Name: ChgOffDate, dtype: object

In [47]:
df1 = df[df["ChgOffDate"].isna()==False]
# df1.count() = 162699 max

df1["ChgOffDate2"] = df1["ChgOffDate"].apply( lambda x : dt.datetime.strptime(x, '%d-%b-%y').date())
df1["ChgOffYear"] = df1["ChgOffDate2"].apply(lambda x: cast(dt.date, x).year) 

df2 = df1[df1["DisbursementDate"].isna()==False]
# df2.count() = 162289 max

df2["DisbursementDate2"] = df2["DisbursementDate"].apply( lambda x : dt.datetime.strptime(x, '%d-%b-%y').date())
df2["DisbursementYear"] = df2["DisbursementDate2"].apply(lambda x: cast(dt.date, x).year) 



/tmp/ipykernel_7841/34720787.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["ChgOffDate2"] = df1["ChgOffDate"].apply( lambda x : dt.datetime.strptime(x, '%d-%b-%y').date())
/tmp/ipykernel_7841/34720787.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["ChgOffYear"] = df1["ChgOffDate2"].apply(lambda x: cast(dt.date, x).year)
/tmp/ipykernel_7841/34720787.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

In [48]:
df2["ChgOffYear"].describe()

count    162289.000000
mean       2008.047335
std           4.936243
min        1988.000000
25%        2007.000000
50%        2009.000000
75%        2011.000000
max        2026.000000
Name: ChgOffYear, dtype: float64

In [49]:
df2["DisbursementYear"].describe()

count    162289.000000
mean       2003.665030
std           5.610377
min        1969.000000
25%        2003.000000
50%        2005.000000
75%        2007.000000
max        2068.000000
Name: DisbursementYear, dtype: float64

In [50]:
#____________________________________________________________
# delete some dates :
def remove_columns(dataframe : pd.DataFrame) -> pd.DataFrame :
    X = dataframe.drop('LoanNr_ChkDgt', axis=1)
    X = X.drop('Name', axis=1)
    X = X.drop('City', axis=1)
    # State included
    X = X.drop('Zip', axis=1)
    X = X.drop('Bank', axis=1)
    X = X.drop('BankState', axis=1)
    # NAICS included (2 first chars )
    X = X.drop('ApprovalDate', axis=1)
    # KEEP an informatin about date
    #____________________________________________________________
    # X = X.drop('ApprovalFY', axis=1) 
    # Term (nombre de mois - à mettre en rapport avec le GrAppv ?) included
    # NoEmp, NewExist , CreateJob, RetainedJob, FranchiseCode, UrbanRural, RevLineCr include 
    # LowDoc (include 'y' only) included
    X = X.drop('ChgOffDate', axis=1)
    X = X.drop('DisbursementDate', axis=1)  
    X = X.drop('DisbursementGross', axis=1)  
    X = X.drop('BalanceGross', axis=1)  # 14 valeurs seulement différentes de zéro  ?
    # MIS_Status : INCLUDED !!! because it is Y
    X = X.drop('ChgOffPrinGr', axis=1)
    # GrAppv : INCLUDED
    return X

X = remove_columns(df_exp)
#y = df['MIS_Status']
#X = df.drop('MIS_Status', axis=1)
X.head(3)

,State,NAICS,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,RevLineCr,LowDoc,MIS_Status,SBA_Appv
0,IN,451120,1997,84,4,2.0,0,0,1,0,N,Y,P I F,"$48,000.00"
1,IN,722410,1997,60,2,2.0,0,0,1,0,N,Y,P I F,"$32,000.00"
2,IN,621210,1997,180,7,1.0,0,0,1,0,N,N,P I F,"$215,250.00"


In [51]:
index =0
for column in X.columns :
    print(f"{index} {column} : {df[column].dtype})")
    index+=1

0 State : object)
1 NAICS : int64)
2 ApprovalFY : object)
3 Term : int64)
4 NoEmp : int64)
5 NewExist : float64)
6 CreateJob : int64)
7 RetainedJob : int64)
8 FranchiseCode : int64)
9 UrbanRural : int64)
10 RevLineCr : object)
11 LowDoc : object)
12 MIS_Status : object)
13 SBA_Appv : object)


# Analyse des colonnes

In [60]:
# Missing values strategy :
import conversion_functions as conv

column_name = X.columns[0]
#X[column_name].info()
X.info()


<class 'pandas.core.frame.DataFrame'>
Index: 897167 entries, 0 to 899163
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   State          897154 non-null  object 
 1   NAICS          897167 non-null  int64  
 2   ApprovalFY     897167 non-null  object 
 3   Term           897167 non-null  int64  
 4   NoEmp          897167 non-null  int64  
 5   NewExist       897033 non-null  float64
 6   CreateJob      897167 non-null  int64  
 7   RetainedJob    897167 non-null  int64  
 8   FranchiseCode  897167 non-null  int64  
 9   UrbanRural     897167 non-null  int64  
 10  RevLineCr      892647 non-null  object 
 11  LowDoc         894589 non-null  object 
 12  MIS_Status     897167 non-null  object 
 13  SBA_Appv       897167 non-null  object 
dtypes: float64(1), int64(7), object(6)
memory usage: 102.7+ MB


In [61]:
X.isna().sum()

State              13
NAICS               0
ApprovalFY          0
Term                0
NoEmp               0
NewExist          134
CreateJob           0
RetainedJob         0
FranchiseCode       0
UrbanRural          0
RevLineCr        4520
LowDoc           2578
MIS_Status          0
SBA_Appv            0
dtype: int64

In [59]:
import numpy as np 

X2 = pd.DataFrame(X)

X2[column_name].fillna('ZZ')
X2[column_name] = X2[column_name].apply(lambda x : conv.get_state_code(x))
#X2[column_name] = X2[column_name].astype("Int64").dtypes

X2[column_name].info()
X2[column_name].unique()

<class 'pandas.core.series.Series'>
Index: 897167 entries, 0 to 899163
Series name: State
Non-Null Count   Dtype
--------------   -----
897167 non-null  int64
dtypes: int64(1)
memory usage: 13.7 MB


array([ 914, 1511,  612,  320, 1410, 1403,  912, 1809, 2024, 2201, 2014,
        118, 1314, 1315, 1301,  301, 1903, 1201,  901, 1508, 1125, 1319,
       1425, 1304, 1601, 1518, 1305, 1119, 1309,  111, 2301,  315, 1320,
       2325, 2120, 1408, 2322,  904,  126, 1422, 2309, 1413,  701, 1404,
       2220,  112, 1405, 1904,  809,  405,  403, 4633])

In [62]:
X2.isna().sum()

State               0
NAICS               0
ApprovalFY          0
Term                0
NoEmp               0
NewExist          134
CreateJob           0
RetainedJob         0
FranchiseCode       0
UrbanRural          0
RevLineCr        4520
LowDoc           2578
MIS_Status          0
SBA_Appv            0
dtype: int64

______________________

In [54]:
numeric = X.drop("State", axis=1)
numeric = numeric.drop("NAICS", axis=1)
numeric = numeric.drop("RevLineCr", axis=1)
numeric = numeric.drop("LowDoc", axis=1)

numeric["MIS_Status"] = numeric["MIS_Status"].apply(lambda x : 0 if x=="P I F" else 1 )
numeric.head()
#numeric["MIS_Status"] = numeric[(numeric["MIS_Status"] == "P I F") | 

,ApprovalFY,Term,NoEmp,NewExist,CreateJob,RetainedJob,FranchiseCode,UrbanRural,MIS_Status,SBA_Appv
0,1997,84,4,2.0,0,0,1,0,0,"$48,000.00"
1,1997,60,2,2.0,0,0,1,0,0,"$32,000.00"
2,1997,180,7,1.0,0,0,1,0,0,"$215,250.00"
3,1997,60,2,1.0,0,0,1,0,0,"$28,000.00"
4,1997,240,14,1.0,7,7,1,0,0,"$229,000.00"


In [55]:
import matplotlib.pyplot as plt

data_corr = pd.to_numeric(numeric, errors="coerce")
data_corr.assign(Value= data_corr).dropna()

plt.matshow(data_corr)
plt.show()


TypeError: arg must be a list, tuple, 1-d array, or Series

In [ ]:


#set( list(X["State"].value_counts().keys())) # => no 'AA' or 'ZZ' value
#____________________________________________________________________________________
# list_values = list(X["NAICS"].value_counts().keys()) # => no 'AA' or 'ZZ' value
# short_list = set(list_values) 
# set_values = set(map (lambda x : conv.get_industry_code(x) ,short_list))
# set_values
#____________________________________________________________________________________
#set(list(X["Term"].value_counts().keys())) # => from 0 to 569
#____________________________________________________________________________________
#set(list(X["NewExist"].value_counts().keys())) # => from 0.0, 1.0 or 2.0
#____________________________________________________________________________________
#set(list(X["CreateJob"].value_counts().keys())) # => from 0. to 8800
#____________________________________________________________________________________
#set(list(X["RetainedJob"].value_counts().keys())) # => from 0 to 9500
#____________________________________________________________________________________
# X["FranchiseCode"].describe()
# X["FranchiseCode"].quantile( [0.9, 0.92, 0.94, 0.96, 0.98, 1.0])
# => 0 1 ou numéro de franchise ( entre 0 et 100 000)
#____________________________________________________________________________________
#X["UrbanRural"].value_counts().keys()
# => 0 1 ou 2
#____________________________________________________________________________________
X["RevLineCr"].value_counts().keys()